In [ ]:
#script using the prince library to do CA on the persons and their biographical properties: https://maxhalford.github.io/prince/

In [12]:
import prince

import sqlite3 as sql
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import plotly.express as px
import prince
from sklearn.preprocessing import OneHotEncoder


In [13]:
### Define the path to the file (a string in a variable 'db_file')
db_file = '../database/cortona_week.db'

In [14]:
### Create a connection to the database
# If the database does not exist it will be created
try:
    cn = sql.connect(db_file)

except Exception as e:
    print(e)


  


In [15]:
### SQL Queries

#only information from persons are aggregated that have attended the cortona events in question
# (events 1-8 until year 1995)

#This query extracts information from the tabel Person if pk_person is found
#in EITHER the table Study OR the table Pursuit thus guaranteeing that we only work with persons for which additional biographical data is known
q_persons = """
SELECT
    p.pk_person,
    p.name,
    p.gender,
    p.birth_date,
    p.nationality
FROM
    Person p
WHERE
    p.pk_person IN (
        SELECT s.fk_person
            FROM Study s
        UNION
        SELECT p2.fk_person
            FROM Pursuit p2
    )
AND
    p.pk_person IN (
        SELECT 
            p4.fk_person
        FROM
            Participation p4
        WHERE
            p4.fk_event < 9
    )
"""
q_all = """
SELECT 
	p2.fk_person,
    o.meta_category,
    NULL,
    o2.meta_category,
    p3.metacategoryCountry
FROM
	Pursuit p2
FULL JOIN
    Organization o
ON
    p2.fk_organization = o.pk_organization

FULL JOIN
    Occupation o2
ON
    p2.fk_occupation = o2.pk_occupation
    
FULL JOIN
    Place p3
ON
    o.fk_place = p3.pk_place

WHERE
    p2.fk_person IN (
        SELECT 
            p4.fk_person
        FROM
            Participation p4
        WHERE
            p4.fk_event < 9
    )

    
UNION
SELECT
    s.fk_person,
    o.meta_category,
    s2.meta_category,
    NULL,
    p3.metacategoryCountry
FROM
    Study s
JOIN
    Subject s2
ON
    s.fk_subject = s2.pk_subject
JOIN
    Organization o
ON
    s.fk_organization = o.pk_organization
JOIN
    Place p3
ON
    o.fk_place = p3.pk_place
WHERE
     s.fk_person IN (
        SELECT 
            p4.fk_person
        FROM
            Participation p4
        WHERE
            p4.fk_event < 12
    )


"""


q_pursuit = """
SELECT 
	p2.fk_person,
    o.meta_category,
    o2.meta_category,
    p3.country
FROM
	Pursuit p2
FULL JOIN
    Organization o
ON
    p2.fk_organization = o.pk_organization

FULL JOIN
    Occupation o2
ON
    p2.fk_occupation = o2.pk_occupation
FULL JOIN
    Place p3
ON
    o.fk_place = p3.pk_place
WHERE
     p2.fk_person IN (
        SELECT 
            p4.fk_person
        FROM
            Participation p4
        WHERE
            p4.fk_event < 9
    )
"""

q_study = """
SELECT
    s.fk_person,
    o.meta_category,
    s2.meta_category,
    p3.country
FROM
    Study s
JOIN
    Subject s2
ON
    s.fk_subject = s2.pk_subject
JOIN
    Organization o
ON
    s.fk_organization = o.pk_organization
JOIN
    Place p3
ON
    o.fk_place = p3.pk_place
WHERE
     s.fk_person IN (
        SELECT 
            p4.fk_person
        FROM
            Participation p4
        WHERE
            p4.fk_event < 12
    )
"""


In [16]:
### Creates container for results
cur = cn.cursor()
cur

In [17]:
### Execute the SQL queries and save the results
cur.execute(q_persons)
data_persons = cur.fetchall()

cur.execute(q_pursuit)
data_pursuit= cur.fetchall()

cur.execute(q_study)
data_study = cur.fetchall()

cur.execute(q_all)
data_all= cur.fetchall()


In [18]:
#dataframe containing the following properties from class PERSON: primary key, name, gender, birth date, nationality
pd_persons = pd.DataFrame(data_persons, columns = ['pkPerson', 'name','gender','birthDate','nationality'])

#dataframe containing the following properties from class PURSUIT: key of person, meta-category of organization, meta-category of occupation, country 
pd_pursuit = pd.DataFrame(data_pursuit, columns = ['pkPerson', 'metacategoryOrganization' , 'metacategoryOccupation', 'countryPursuit'])

#dataframe containing the following properties from class STUDY: key of person,meta-category of subject, meta-category of organization, country 
pd_study = pd.DataFrame (data_study, columns = ['pkPerson','studyMetaCategory','organizationMetacategory','countryStudy'])

#dataframe containing the following properties from both class PURSUIT AND STUDY:
#key of person, meta-category of Organization of pursuit, meta-category of subject from study, meta-category of occupation from pursuit
#meta-category of place (=amounts to continent) for BOTH pursuit and study
pd_pursuit_study = pd.DataFrame(data_all, columns = ['pkPerson', 'metacategoryOrganizationPursuit' ,
                                                     'studyMetaCategory', 'occupationMetacategory','countryPursuitStudy'])



pd_persons=pd_persons.sort_values(by=['pkPerson'])
pd_pursuit=pd_pursuit.sort_values(by=['pkPerson'])
pd_study=pd_study.sort_values(by=['pkPerson'])
pd.set_option('display.max_rows', 10)

#display(pd_pursuit_study)
#display(pd_study)

'''
some code to test something. Should be removed once I remember what it was I was testing for and whether it is important^^.
all_person= pd.concat([pd_pursuit['pkPerson'],pd_study['pkPerson']])
all_person=all_person.sort_values()
pd.set_option('display.max_rows', 500)
#display(all_person)

#print(pd_pursuit['pkPerson'].nunique())
#print(pd_study['pkPerson'].nunique())

#print(pd_persons['pkPerson'])
all_person_unique=all_person.unique()

max_persons= pd_persons['pkPerson'].nunique()

not_in_mask= ~pd_persons['pkPerson'].isin(all_person)
#print(not_in_mask)
#print(pd_persons['pkPerson'][not_in_mask])
#print(all_person.nunique())
#print(pd_persons['pkPerson'].nunique())

pd.set_option("display.max_rows", 10)
#display(pd_persons)
#display(pd_pursuit)
#display(pd_study)
'''

'\nsome code to test something. Should be removed once I remember what it was I was testing for and whether it is important^^.\nall_person= pd.concat([pd_pursuit[\'pkPerson\'],pd_study[\'pkPerson\']])\nall_person=all_person.sort_values()\npd.set_option(\'display.max_rows\', 500)\n#display(all_person)\n\n#print(pd_pursuit[\'pkPerson\'].nunique())\n#print(pd_study[\'pkPerson\'].nunique())\n\n#print(pd_persons[\'pkPerson\'])\nall_person_unique=all_person.unique()\n\nmax_persons= pd_persons[\'pkPerson\'].nunique()\n\nnot_in_mask= ~pd_persons[\'pkPerson\'].isin(all_person)\n#print(not_in_mask)\n#print(pd_persons[\'pkPerson\'][not_in_mask])\n#print(all_person.nunique())\n#print(pd_persons[\'pkPerson\'].nunique())\n\npd.set_option("display.max_rows", 10)\n#display(pd_persons)\n#display(pd_pursuit)\n#display(pd_study)\n'

In [19]:
#one-hot encoding (=creates a dummy variable for each different categorical value which is then either True or False for each person)

#testing with all properties I have available

##on-hot encode all the data from the classes Pursuit and Study
pd_all_to_encode=pd_pursuit_study[['pkPerson','metacategoryOrganizationPursuit', 'studyMetaCategory','countryPursuitStudy','occupationMetacategory']]
pd_all_hot_encode=pd.get_dummies(pd_all_to_encode,dummy_na=True)
pd.set_option('display.max_rows', 10)

display(pd_all_hot_encode)

#sum up according to pkPerson
grouped_pd_all_hot_encode=pd_all_hot_encode.groupby('pkPerson').sum().reset_index()

##on-hot encode all the data (except birth/death date) from class Person
pd_person_to_encode = pd_persons[['pkPerson','gender','nationality']]
pd_person_hot_encode=pd.get_dummies(pd_person_to_encode,dummy_na=False)


##combine the two one-hot encoded data sets base on pkPerson
output=grouped_pd_all_hot_encode.merge(pd_person_hot_encode, left_on='pkPerson', right_on='pkPerson', how = 'inner')
output = output.loc[:, output.columns != 'pkPerson']
output_full = output.replace({True: 1, False: 0})


,pkPerson,metacategoryOrganizationPursuit_Firma,metacategoryOrganizationPursuit_Medizin,metacategoryOrganizationPursuit_Publikation,metacategoryOrganizationPursuit_Universität,metacategoryOrganizationPursuit_Wissenschaftliche Vereinigung,metacategoryOrganizationPursuit_alternative Forschungsinstut,metacategoryOrganizationPursuit_gegenkulturelle Gemeinschaft,metacategoryOrganizationPursuit_internationales Netzwerk,metacategoryOrganizationPursuit_künstlerischer Ort,...,occupationMetacategory_Management,occupationMetacategory_Mitarbeiter:in,occupationMetacategory_Mitglied,occupationMetacategory_Public official,occupationMetacategory_Seminarleiter:in,occupationMetacategory_Therapeut:in,occupationMetacategory_Universitätsposten,occupationMetacategory_redaktioneller Beruf,occupationMetacategory_spiritueller Leiter:in,occupationMetacategory_nan
0,1,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,1,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,3,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,3,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,5,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,357,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
429,357,False,False,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,False,False
430,360,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
431,360,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


/tmp/ipykernel_141/1282003648.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  output_full = output.replace({True: 1, False: 0})


In [20]:

#initalize MCA model for data set that is ALREADY one-hot encoded
mca_no_one_hot = prince.MCA(n_components=8,
    n_iter=3,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42,
    one_hot=False)

#fit the full data set
mca_no_one_hot = mca_no_one_hot.fit(output_full)
mca_no_one_hot.eigenvalues_summary

mca_no_one_hot.plot(
    output_full,
    x_component=0,
    y_component=3,
    show_column_markers=True,
    show_row_markers=True,
    show_column_labels=False,
    show_row_labels=False
)
#mca_no_one_hot.eigenvalues_summary


alt.LayerChart(...)

In [21]:
mca_no_one_hot.eigenvalues_summary

,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,0.313,5.69%,5.69%
1,0.300,5.46%,11.16%
2,0.263,4.79%,15.95%
3,0.227,4.13%,20.08%
4,0.222,4.05%,24.12%
5,0.196,3.56%,27.69%
6,0.180,3.27%,30.96%
7,0.166,3.02%,33.98%


In [22]:
mca_no_one_hot.column_contributions_.style.format('{:.0%}')

,0,1,2,3,4,5,6,7
metacategoryOrganizationPursuit_Firma,0%,5%,5%,1%,7%,20%,0%,1%
metacategoryOrganizationPursuit_Medizin,0%,0%,0%,0%,0%,1%,0%,0%
metacategoryOrganizationPursuit_Publikation,0%,0%,0%,32%,0%,0%,0%,1%
metacategoryOrganizationPursuit_Universität,2%,7%,0%,0%,0%,2%,0%,2%
metacategoryOrganizationPursuit_Wissenschaftliche Vereinigung,0%,0%,0%,0%,0%,1%,0%,0%
metacategoryOrganizationPursuit_alternative Forschungsinstut,4%,3%,0%,0%,0%,0%,11%,0%
metacategoryOrganizationPursuit_gegenkulturelle Gemeinschaft,1%,1%,6%,0%,0%,0%,1%,0%
metacategoryOrganizationPursuit_internationales Netzwerk,1%,0%,7%,0%,0%,1%,0%,0%
metacategoryOrganizationPursuit_künstlerischer Ort,1%,0%,0%,0%,2%,2%,0%,4%
metacategoryOrganizationPursuit_politische Vereinigung,2%,1%,0%,0%,0%,0%,3%,3%


In [13]:
#one-hot encoding

#leave birth_date out for now
pd_person_to_encode = pd_persons[['pkPerson','gender', 'nationality']]

pd_person_hot_encode=pd.get_dummies(pd_person_to_encode,dummy_na=False)

display(pd_person_hot_encode)
pd_study_hot_encode=pd.get_dummies(pd_study,dummy_na=False)
#display(pd_study_hot_encode)

grouped_pd_study_hot_encode=pd_study_hot_encode.groupby('pkPerson').sum().reset_index()
#display(grouped_pd_study_hot_encode)

pd_pursuit_hot_encode=pd.get_dummies(pd_pursuit,dummy_na=False)
#display(pd_pursuit_hot_encode)

grouped_pd_pursuit_hot_encode=pd_pursuit_hot_encode.groupby('pkPerson').sum().reset_index()
#display(grouped_pd_pursuit_hot_encode)





output=pd_pursuit_hot_encode.merge(grouped_pd_study_hot_encode, left_on='pkPerson', right_on='pkPerson', how = 'inner')
pd.set_option("display.max_columns", 30)
pd.set_option("display.max_rows", 35)
#display(output)

#grouped_output=output.groupby('pkPerson').sum().reset_index()
#display(grouped_output)
#output_full=output.merge(pd_person_hot_encode, left_on='pkPerson', right_on='pkPerson', how='outer')
#display(output_full)
#output_full_encoded=pd.get_dummies(output_full,dummy_na=True)
#display(output_full_encoded)

output=output.fillna(0)
output = output.replace({True: 1, False: 0})
output=output.groupby('pkPerson').sum().reset_index()


output_full=output.merge(pd_person_hot_encode, left_on='pkPerson', right_on='pkPerson', how='inner')
output_full=output_full.fillna(0)
output_full = output_full.replace({True: 1, False: 0})
output_full=output_full.groupby('pkPerson').sum().reset_index()
output_full=output_full.astype('float')

pd.set_option("display.max_columns", 5)
pd.set_option("display.max_rows", 5)

is_infinite = np.isinf(output_full.values)

# Check if any infinity values exist
res = is_infinite.any()
print(res)
is_nan = np.isnan(output_full.values)

# Check if any nan values exist
res = is_nan.any()
print(res)


output_full[output_full > 1] = 1
pd.set_option("display.max_columns", 5)
pd.set_option("display.max_rows", 5)
#display(output_full)
#output_full = output_full.replace({1: True, 0: False})
output_full = output_full.loc[:, output_full.columns != 'pkPerson']
#display(output_full)
output_full.to_csv('output_for_MCA.csv',index=True) 


print(output_full.columns[output_full.sum(axis=0)==0])
print(np.sum(output_full.sum(axis=1)==0)) 

print(np.sum(output_full.sum(axis=0)==0)) 

output_full_dropped = output_full.drop(columns=output_full.columns[output_full.sum(axis=0)==0])

mca_no_one_hot = prince.MCA(one_hot=False)
mca_no_one_hot = mca_no_one_hot.fit(output_full_dropped)


mca_no_one_hot.plot(
    output_full_dropped,
    x_component=0,
    y_component=1,
    show_column_markers=True,
    show_row_markers=True,
    show_column_labels=False,
    show_row_labels=False
)
#test_study=pd.get_dummies(pd_study,dummy_na=True)
#display(test_study)


,pkPerson,gender_m,...,nationality_USA,nationality_Österreich
0,1,True,...,False,False
1,3,False,...,False,False
...,...,...,...,...,...
162,357,True,...,False,False
163,360,True,...,False,False


False
False
Index(['metacategoryOrganization_künstlerischer Ort',
       'metacategoryOccupation_Heiler:in', 'countryPursuit_Indien',
       'countryPursuit_Japan', 'countryPursuit_Slowenien',
       'nationality_Frankreich', 'nationality_Irland', 'nationality_Kuba',
       'nationality_Palästina', 'nationality_Slowenien', 'nationality_Tibet'],
      dtype='object')
0
11


/tmp/ipykernel_4908/1432947590.py:38: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  output = output.replace({True: 1, False: 0})
/tmp/ipykernel_4908/1432947590.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  output_full = output_full.replace({True: 1, False: 0})


,metacategoryOrganization_Firma,metacategoryOrganization_Medizin,...,nationality_USA,nationality_Österreich
0,0.0,0.0,...,0.0,0.0
1,0.0,0.0,...,0.0,0.0
...,...,...,...,...,...
99,0.0,0.0,...,0.0,0.0
100,0.0,0.0,...,0.0,0.0


alt.LayerChart(...)